![alttext](https://github.com/UrbsLab/STREAMLINE/blob/main/Pictures/STREAMLINE_LOGO.jpg?raw=true)

STREAMLINE is an end-to-end automated machine learning (AutoML) pipeline that empowers anyone to easily run, interpret, and apply a rigorous and customizable analysis for data mining or predictive modeling. Currently limited to binary classification in tabular data.

--------------

# README
* This notebook runs all primary elements STREAMLINE. We recommend users review the STREAMLINE documentation for details.

## What to expect running this notebook 'as-is'?
* This notebook has been initially set up to run 'as-is' on two 'demo' datasets downloaded from the UCI repository: the original HCC dataset, and the same dataset with two covariate features removed. Suggested default run parameters suitible for most users are included. 

* After training models and testing evaluations are complete, these models are applied to a 'replication' dataset for another round of evaluations. Since no true replication data was available for this example, we used a copy of the original HCC dataset simply to demonstrate the entire pipeline in action. Since these 'replication' analyses are conducted on data that includes the original training data, you will observe better model performance in that secondary evaluation. In true replication data you would generally expect as good or worse performance. 

* Notebook run parameters have initially been set up to run only three of the available modeling algorithms (logistic regression, decision tree, and naive bayes), with 3-fold CV so that it runs completely in about 6-7 minutes. 

* As the notebook run completes, the PDF summary(s) and zipped output folder will automatically downloaded to your computer. You will likely be propted to accept the download on your first run. 

## Run Instructions for this Notebook
* For Demo: 
  * Leave all run parameter cells (below) unchanged and choose 'RunAll' under 'Runtime' tab in Colab Notebook
  * You can also change non-dataset related run parameters below to run the demo with different settings

* Custom Dataset Run: 

  * Easy Mode:
    * Set (demo_run = False) and (use_data_prompt = True), then edit the remaining run parameters below as desired, excluding dataset parameters
    * Choose 'RunAll' under 'Runtime' tab in Colab Notebook.
    * When notebook runs, the user will first be prompted to load datasets (in one, and specify other critical dataset parameters. This includes:
      * (Required) The output folder name for the current STREAMLINE experiment
      * (Required) Selecting dataset(s) for STREAMLINE analysis
      * Specifying (without single or double quotation marks):
        * (Required) class column label
        * (If present) instanceID column label
        * (If present) Match column label (for covariate matched data)      
      * (Required) If replication data is available
        * (If available) Selecting dataset(s) for replication evaluation
        * (If available) Specifying the name of the target dataset (including file extension) whos models the replication data will be evaluating

  * Manual Mode:
    * Set (demo_run = False) and (use_data_prompt = False)
    * Manually create a folder in /content/ (i.e. locally in the google colab workspace) 
    * Edit all run parameters accordingly, including target dataset filepaths
    * Choose 'RunAll' under 'Runtime' tab in Colab Notebook.
* Before running the STREAMLIN notebook again we generally recommend selecting "Disconnect and delete runtime' under 'Runtime' tab in Colab Notebook.

--------------

# STREAMLINE RUN PARAMETERS
The first two paramters below are specific to this Google Colab notebook.

In [1]:
demo_run = True # Leave (True) to run the demo dataset, make (False) to have new temporary folder created to upload datasets
use_data_prompt = True # Generally leave as (True) unless you want to dissable the prompts (that allows users to run their own data without changing the dataset run parameters in this notebook)

### Run Parameters: Target Datasets (Ignore if Running Demo Data)
* No need to edit unless (demo_run = False) and (use_data_prompt = False).
* Update these parameters to run STREAMLINE on a different folder of datasets. Any folder of datasets to be analyzed should include one or more datasets saved as .txt or .csv files. See documentation for dataset formatting requirements. 

* All datasets should have the same header names for the class, instance, and match labels (note instance and match labels are optional). 

In [2]:
if not demo_run: # Leave this command as is.

    # File path to the folder containing dataset(s) to be analyzed (must include one or more .txt, .tsv, or .csv datasets)
    data_path = "/content/UserData" # (str) Data Folder Path

    # Output foder path: where to save pipeline outputs (must be updated for a given user)
    output_path = '/content/UserOutput' # (str) Ouput Folder Path (folder will be created by STREAMLINE automatically)

    # Unique experiment name - folder created for this analysis within output folder path
    experiment_name = 'my_experiment'  # (str) Experiment Name (change to save a new STREAMLINE run output folder instead of overwriting previous run)

    # Data Labels
    class_label = 'Class' # (str) i.e. class outcome column label
    instance_label = 'InstanceID' # (str) If data includes instance labels, given respective column name here, otherwise put 'None'
    match_label = None # (str or None) Only applies when M selected for partition-method; indicates column label with matched instance ids' 

    # Option to manually specify feature names to leave out of analysis, or which to treat as categorical (without using built in variable type detector)
    ignore_features = None # list of column names (given as string values) to exclude from the analysis (only insert column names if needed, otherwise leave empty)
    categorical_feature_headers = None # empty list for 'auto-detect' otherwise list feature names (given as string values) to be treated as categorical. Only impacts algorithms that can take variable type into account.

### Run Parameters: General
* Optionally update these general parameters used throughout all/most phases of the pipeline.

In [3]:
# Cross Validation (CV)
n_splits = 3  # (int, > 1) Number of training/testing data partitions to create - and resulting number of models generated using each ML algorithm
partition_method = 'Stratified' # (str) for Stratified, Random, or Group, respectively

# Cutoffs
categorical_cutoff = 10 # (int) Bumber of unique values after which a variable is considered to be quantitative vs categorical
sig_cutoff = 0.05 # (float, 0-1) Significance cutoff used throughout pipeline
# Set Random Seed for Reproducible Analysis
random_state = 42 # (int) Sets a specific random seed for reproducible results

### Run Parameters: Data Processing
* Optionally, update these parameters to decide what analyses are run and outputs are produced by STREAMLINE in the exploratory analysis phase.

In [4]:
# Analysis options to turn on or off in the exploratory analysis ("Describe" = basic descriptive data stats, "Differentiate")
exploration_list = ["Describe", "Univariate Analysis", "Feature Correlation"] # (list of strings) Options:["Describe", "Differentiate", "Univariate Analysis"]

# Control what exploratory analysis plots get generated
plot_list = ["Describe", "Univariate Analysis", "Feature Correlation"] # (list of strings) Options:["Describe", "Univariate Analysis", "Feature Correlation"]

# univariate analysis plots (note: univariate analysis still output by default)
top_features = 20 # (int) Number of top features to report in notebook for univariate analysis

featureeng_missingness = 0.5 # (float, 0-1) Percentage of missing after which categorical featrure identifier is generated.
cleaning_missingness = 0.5 # (float, 0-1) Percentage of missing after instance and feature removal is performed.
correlation_removal_threshold = 0.8

### Run Parameters: Scaling and Imputing
* Optionally update these parameters to turn specific data preprocessing options on or off.

In [5]:
# Data Transformation (i.e. scaling) - important for running and interpreting built-in feature importance estimates for certain ML modeling algorithms 
scale_data = True # (bool, True or False) Perform data scaling

# Missing Data Imputation Options
impute_data = True # (bool, True or False) Perform missing value data imputation? (required for most ML algorithms if missing data is present)
multi_impute = True # (bool, True or False) Applies multivariate imputation to quantitative features, otherwise uses mean imputation

# Option for how cross validation datasets are saved (for external use)
overwrite_cv = True # (bool, True or False) Overwrites earlier cv datasets with new scaled/imputed ones

### Run Parameters: Feature Importance Estimation
* Optionally update these parameters to decide which filter-based feature importance estimation algorithms to apply (currently only mutual information and MultiSURF are options). 

In [6]:
# Available Filter-based Feature Importance/Selection Algorithms
do_mutual_info = True # (bool, True or False) Do mutual information analysis
do_multisurf = True # (bool, True or False) Do multiSURF analysis

# Additional MultiSURF Options
use_TURF = False # (bool, True or False) Use TURF wrapper around MultiSURF
TURF_pct = 0.5 # (float, 0.01-0.5) Proportion of instances removed in an iteration (also dictates number of iterations)
instance_subset = 2000 # (int) Sample subset size to use with MultiSURF (since MultiSURF's compute time scales quadratically with instance count)
njobs = -1 # (int) Number of cores dedicated to running algorithm; setting to -1 will use all available cores when run locally

### Run Parameters: Feature Selection
* Optionally update these parameters to control how 'collective' feature selection is conducted prior to modeling. 

  * When 'filter_poor_features' = False, all features will be used in the modeling phase. 

  * When 'filter_poor_features' = True: 
    * And 'max_features_to_keep' = 'None', all features with a score <= 0 from all active feature importance algorithm will be removed, but the rest kept. 

    * And 'max_features_to_keep' = n (where n is a 'value' less than the total number of features in the dataset), first all features with a score <= 0 from all active feature importance algorithm will be removed, then the top n scoring (non-redundant) features from each algorithm will be kept. 

In [7]:
# Turn feature selection on or off 
filter_poor_features = True # (bool, True or False) Filter out the worst performing features prior to modeling

# Control maximum number of features to keep out of total features in dataset.
max_features_to_keep = 2000 # (int) Maximum features to keep. 'None' if no max

# Controls the feature importance estimation plots generation
top_features = 40 # (int) Number of top features to illustrate in figures
export_scores = True # (bool, True or False) Export figure summarizing average feature importance scores over cv partitions

### Run Parameters: Modeling
* Optionally update these parameters to control what modeling algorithms are run, as well as other options relevant to the modeling phase. The 16 Classification algorithms currently available in STREAMLINE include:

  * Naive Bayes (NB)
  * Logistic Regression (LR)
  * Elastic Net (EN)
  * Decision Tree (DT)
  * Random Forest (RF)
  * Gradient Boosting (GB)
  * Extreame Gradient Boosting (XGB)
  * Light Gradient Boosting (LGB)
  * Category Gradient Boosting (CGB)
  * Support Vector Machines (SVM)
  * Artificial Neural Networks (ANN)
  * K-Nearest Neighbors (KNN)
  * Genetic Programming, i.e. symbolic classification (GP)
  * Educational Learning Classifier System (eLCS)
  * 'X' Classifier System (XCS)
  * Extended Supervised Tracking Classifier System (ExSTraCS)

The last 3 algorithms above are rule-based ML approaches implemented by our research group. Their are currently suspected bugs in eLCS and XCS so they have been turned off when using default settings.


In [8]:
# Machine Learning Algorithms to Run (Setting 'algorithms' to 'None' rather than a list will run all algorithms except those specified in 'exclude')
algorithms = ["NB", "LR", "DT"] # (list of strings) Options: ["NB","LR","EN","DT","RF","GB","XGB","LGB","CGB","SVM","ANN","KNN","GP","eLCS","XCS","ExSTraCS"]

# ML Model Algorithm to exclude
exclude = ['eLCS', 'XCS'] # (list of strings) Options: ["NB","LR","EN","DT","RF","GB","XGB","LGB","CGB","SVM","ANN","KNN","GP","eLCS","XCS","ExSTraCS"]

# Other Analysis Parameters
training_subsample = 0  # (int) For long running algorithms, option to subsample training set (0 for no subsample) Limit Sample Size Used to train algorithms that do not scale up well in large instance spaces (i.e. XGB,SVM,KN,ANN,and LR to a lesser degree) and depending on 'instances' settings, ExSTraCS, eLCS, and XCS)
use_uniform_FI = True # (bool, True or False) Overides use of any available feature importances estimate methods from models, instead using permutation_importance uniformly
primary_metric = 'balanced_accuracy' # (str) Must be an available metric identifier from (https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
metric_direction = 'maximize' # (str, either of 'maximize' or 'minimize')

# Hyperparameter Sweep Options
n_trials = 200   # (int or None) Number of bayesian hyperparameter optimization trials using optuna
timeout = 900    # (int or None) Seconds until hyperparameter sweep stops running new trials (Note: it may run longer to finish last trial started)
export_hyper_sweep_plots = True # (bool, True or False) Export hyper parameter sweep plots from optuna

# Learning classifier system algorithm options (ExSTraCS, eLCS, XCS)
do_lcs_sweep = False # (bool, True or False) Do LCS hyperparam tuning or use below params
lcs_nu = 1                 # (int, 0-10) Fixed LCS nu param
lcs_iterations = 200000    # (int, > data sample size) Fixed LCS # learning iterations param
lcs_N = 2000               # (int) > 500) Fixed LCS rule population maximum size param
lcs_timeout = 1200     # (int) Seconds until hyperparameter sweep stops for LCS algorithms (evolutionary algorithms often require more time for a single run)

### Run Parameters: Evaluation Figure Generation (Stats Phase)
* Optionally update these parameters to control aspects of model evaluation figure generation. Note that all STREAMLINE performance metric evaluations and figures are generated with respect to the hold out testing data in this phase.

In [9]:
# ROC and PRC Plot Generation for Individual Algorithms (including separate curves for each CV model trained)
plot_ROC = True    # (bool, True or False) Plot ROC curves individually for each algorithm including all CV results and averages
plot_PRC = True    # (bool, True or False) Plot PRC curves individually for each algorithm including all CV results and averages

# Feature Importance Plots (for each algorithm)
plot_FI_box = True # (bool, True or False) Plot feature importance boxplots for each algorithm

# Box Plots Comparing Algorithm Performance (for each evaluation metric)
plot_metric_boxplots = True # (bool, True or False) Plot box plot summaries comparing algorithms for each metric

# Composite Feature Importance Plot Setting
metric_weight = 'balanced_accuracy' # (str, balanced_accuracy or roc_auc) ML model metric used as weight in composite FI plots (only supports balanced_accuracy or roc_auc as options) Recommend setting the same as primary_metric if possible.

# Feature Importance Plot Setting (impacts individual feature importance plots for each algorith and the composite feature importance plot)
top_model_features = 40  # (int) Number of top features in model to illustrate in feature importance figures

### Run Parameters: Replication Data (Ignore if Running Demo Data)
* Don't edit unless (demo_run = False) and (use_data_prompt = False).

* Update these parameters to run STREAMLINE's 'apply' phase where all models trained in the earlier phases are evaluated on the same hold-out replication data (recommended when available).

* Multiple replication datasets (e.g. data collected from different sites) can be included in the replication data folder and STREAMLINE will apply replication analyses to each individually. 

In [10]:
if not demo_run: # Leave this command as is.

   # Turns the replication data analysis phase on or off
    applyToReplication = True # (bool, True or False) Leave false unless you have a replication dataset handy to further evaluate/compare all models in uniform manner

    # File path to the folder containing the replication dataset(s) to be evaluated using previously trained models (.txt, .tsv, or .csv datasets))
    rep_data_path = "/content/UserRepData" # (txt) Name of folder with replication Dataset(s)

    # File path to one of the individual datasets used to train models within STREAMLINE
    dataset_for_rep = "/content/UserData/hcc-data_example.csv" # (txt) Path and name of an individual dataset used to generate the models being evaluated with replication data

### Run Parameters: Output File Cleanup
* Optionally update these parameters to delete temporary files in output folder (generally recommended to leave these as True).

In [11]:
del_time = True  # (bool, True or False) Delete individual run-time files (but save summary)
del_old_cv = True # (bool, True or False) Delete any of the older versions of CV training and testing datasets if overwrite_cv was set to False (preserves training and testing datasets used in pipeline)

-------------

### Users - Don't edit notebook below this cell.

--------------

# DEMO DATA ANALYSIS SETUP 


### Demo Data Run Parameters:
* Set up for Demo Run (never edit the code cell below)


In [12]:
if demo_run: # Leave this command as is.

    # File path to the folder containing dataset(s) to be analyzed (must include one or more .txt, .tsv, or .csv datasets)
    data_path = "/content/STREAMLINE/DemoData" # (str) Data Folder Path

    # Output foder path: where to save pipeline outputs (must be updated for a given user)
    output_path = '/content/DemoOutput' # (str) Ouput Folder Path (folder will be created by STREAMLINE automatically)

    # Unique experiment name - folder created for this analysis within output folder path
    experiment_name = 'hcc_demo'  # (str) Experiment Name (change to save a new STREAMLINE run output folder instead of overwriting previous run)

    # Data Labels
    class_label = 'Class' # (str) i.e. class outcome column label
    instance_label = 'InstanceID' # (str) If data includes instance labels, given respective column name here, otherwise put 'None'
    match_label = None # (str or None) Only applies when M selected for partition-method; indicates column label with matched instance ids' 

    # Option to manually specify feature names to leave out of analysis
    ignore_features = None # list of column names (given as string values) to exclude from the analysis (only insert column names if needed, otherwise leave empty)

    # Recommended option to manually specify what features to treat as categorical (None for 'auto-detect', otherwise list feature names (given as string values) to be treated as categorical. 
    # See https://archive.ics.uci.edu/ml/datasets/HCC+Survival for breakdown of feature types: nominal (i.e. categorical), or ordinal, continuous and integer (i.e. quantitative)
    categorical_feature_headers = ['Gender','Symptoms','Alcohol','Hepatitis B Surface Antigen','Hepatitis B e Antigen','Hepatitis B Core Antibody','Hepatitis C Virus Antibody','Cirrhosis','Endemic Countries','Smoking','Diabetes','Obesity','Hemochromatosis','Arterial Hypertension','Chronic Renal Insufficiency','Human Immunodeficiency Virus','Nonalcoholic Steatohepatitis','Esophageal Varices','Splenomegaly','Portal Hypertension','Portal Vein Thrombosis','Liver Metastasis','Radiological Hallmark'] 

    # Turns the replication data analysis phase on or off
    applyToReplication = True # (bool, True or False) Leave false unless you have a replication dataset handy to further evaluate/compare all models in uniform manner

    # File path to the folder containing the replication dataset(s) to be evaluated using previously trained models (.txt, .tsv, or .csv datasets))
    rep_data_path = "/content/STREAMLINE/DemoRepData" # (txt) Name of folder with replication Dataset(s)

    # File path to one of the individual datasets used to train models within STREAMLINE
    dataset_for_rep = "/content/STREAMLINE/DemoData/hcc-data_example.csv" # (txt) Path and name of an individual dataset used to generate the models being evaluated with replication data

-------------

# STREAMLINE RUN CODE


## Dataset Parameters Prompt Setup

In [13]:
def run_prompts(demo_run,use_data_prompt,original_wd):
    import os

    output_path = '/content/UserOutput' # Make separate ouput folder to save STREAMLINE experiment runs on non-demo data

    # Get and save user datasets locally
    custom_data_path = '/content/UserData' # Make local folder for saving user specified target datasets for STREAMLINE anlaysis

    # Check if the directory exists
    if not os.path.exists(custom_data_path):
        # Create the directory if it doesn't exist
        os.makedirs(custom_data_path)
    else:
        # Traverse the directory tree and delete all files and directories
        for root, dirs, files in os.walk(custom_data_path, topdown=False):
            for name in files:
                os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        os.rmdir(custom_data_path)
        os.makedirs(custom_data_path)

    os.chdir(custom_data_path)

    # Ask user for unique experiment name (we do this first so the screen jumps to this prompt)
    experiment_name = input("Enter unique experiment name for output folder (required, no spaces): \n")

    # Have user upload target dataset(s)
    from google.colab import files
    uploaded = files.upload() # Prompt user to select one or more datasets to upload into the 'UserData' folder
    os.chdir(original_wd)

    # Gather other necessary target data information
    class_label = input("Enter header label of class column (required): \n")
    instance_label = input("Enter header label of instance ID or specify None: \n")
    match_label = input("Enter header label of match column or specify None: \n")
    if instance_label == "None":
        instance_label = None
    if match_label == "None":
        match_label = None

    # Ask user whether replication dataset(s) are available
    applyToReplication = eval(input("Is replication data available? Enter True or False (required): \n"))
    custom_rep_data_path = None
    dataset_for_rep = None
    if applyToReplication:
        custom_rep_data_path = '/content/UserRepData' # Make local folder for saving user specified target datasets for STREAMLINE anlaysis

        # Check if the directory exists
        if not os.path.exists(custom_rep_data_path):
            # Create the directory if it doesn't exist
            os.makedirs(custom_rep_data_path)
        else:
            # Traverse the directory tree and delete all files and directories
            for root, dirs, files in os.walk(custom_rep_data_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(custom_rep_data_path)
            os.makedirs(custom_rep_data_path)

        os.chdir(custom_rep_data_path)

        # Have user upload replication dataset(s)
        from google.colab import files
        uploaded = files.upload() # Prompt user to select one or more datasets to upload into the 'UserData' folder
        os.chdir(original_wd)

        #Have user idenify the name of the target dataset used to train the model to which the replication dataset(s) will be applied
        dataset_for_rep = custom_data_path +'/'+ input('Enter the filename (with extension) of the dataset in /UserData/ to indicate which models the replication data will be applied to: \n')

    return custom_data_path,custom_rep_data_path,class_label,instance_label,match_label,dataset_for_rep,applyToReplication,experiment_name

In [14]:
import os

# Get current working directory
original_wd = os.getcwd()

if not demo_run and use_data_prompt:
    # Run User Prompts
    data_path,rep_data_path,class_label,instance_label,match_label,dataset_for_rep,applyToReplication,experiment_name = run_prompts(demo_run,use_data_prompt,original_wd)

In [15]:
# Leave this empty code cell

## Install STREAMLINE and Prerequisites
* Downloads most recent version of STREAMLINE from GitHub and installs other packages required by STREAMLINE. 

In [16]:
#!git clone -b https://github.com/UrbsLab/STREAMLINE.git -q
!git clone -b dev https://github.com/UrbsLab/STREAMLINE.git -q

%cd STREAMLINE

!pip install -r requirements.txt &> /dev/null
!pip install --upgrade scipy>=1.8.0

fatal: destination path 'STREAMLINE' already exists and is not an empty directory.
/Users/harshbandhey/Local/Cedars/Urbslab/STREAMLINE_Dev/streamline
zsh:1: 1.8.0 not found


## Notebook Housekeeping
* Sets up notebook cells to display internal process. 

* Use logging.INFO for higher level output, logging.WARNING for only critical information. Comment to hide all text output.

* You can use run_parallel=True for phases other than modeling, but the advantage is not significant vs the overhead for small jobs.

In [17]:
import logging
FORMAT = '%(levelname)s: %(message)s'
logging.basicConfig(format=FORMAT)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

* Housekeeping code allowing notebook to be run again with the same settings, overwriting a previously run experiment with the same name.

In [18]:
import os
import shutil
if os.path.exists(output_path+'/'+experiment_name):
    shutil.rmtree(output_path+'/'+experiment_name)

## STREAMLINE Workflow
* The code below runs through the analysis phases of STREAMLINE.

## Phase 1: Data Processing Phase
After cell runs, for each target dataset you will see:
* A data count summary
* Class balance barplot
* Feature correlation heatmap
* Top univariate analysis results


In [19]:
from streamline.runners.dataprocess_runner import DataProcessRunner
dpr = DataProcessRunner(data_path, output_path, experiment_name, 
                exploration_list=exploration_list, plot_list=plot_list,
                class_label=class_label, instance_label=instance_label, 
                match_label=match_label, n_splits=n_splits, 
                partition_method=partition_method,
                ignore_features=ignore_features, 
                categorical_features=categorical_feature_headers, 
                top_features=top_features,
                categorical_cutoff=categorical_cutoff, sig_cutoff=sig_cutoff,
                featureeng_missingness=featureeng_missingness,
                cleaning_missingness=cleaning_missingness,
                correlation_removal_threshold=correlation_removal_threshold,
                random_state=random_state, show_plots=True)
dpr.run(run_parallel=False)

INFO: NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'dask_jobqueue'

## Phase 2: Scaling and Imputation
After cell runs, you will see:
* No output other than code progress updates

In [ ]:
from streamline.runners.imputation_runner import ImputationRunner
ir = ImputationRunner(output_path, experiment_name, 
                        scale_data=scale_data, impute_data=impute_data,
                        multi_impute=multi_impute, overwrite_cv=overwrite_cv, 
                        class_label=class_label, instance_label=instance_label, 
                        random_state=random_state)
ir.run(run_parallel=False)

## Phase 3: Feature Importance Evaluation
After cell runs, you will see:
* No output other than code progress updates

In [ ]:
feat_algorithms = []
if do_mutual_info:
    feat_algorithms.append("MI")
if do_multisurf:
    feat_algorithms.append("MS")

In [ ]:
from streamline.runners.feature_runner import FeatureImportanceRunner
f_imp = FeatureImportanceRunner(output_path, experiment_name, 
                                class_label=class_label, 
                                instance_label=instance_label,
                                instance_subset=instance_subset, 
                                algorithms=feat_algorithms, 
                                use_turf=use_TURF, turf_pct=TURF_pct, 
                                random_state=random_state)
f_imp.run(run_parallel=False)

## Phase 4: Feature Selection
After cell runs, for each target dataset and each feature importance algorithm you will see:
* Top feature importance scores
* A barplot of top feature imporance score ranking

In [ ]:
from streamline.runners.feature_runner import FeatureSelectionRunner
f_sel = FeatureSelectionRunner(output_path, experiment_name, 
                               feat_algorithms, class_label=class_label, 
                               instance_label=instance_label,
                               max_features_to_keep=max_features_to_keep, 
                               filter_poor_features=filter_poor_features, 
                               top_features=top_features, 
                               export_scores=export_scores,
                               overwrite_cv=overwrite_cv, 
                               random_state=random_state,
                               show_plots=True)
f_sel.run(run_parallel=False)

## Phase 5: Modeling
After cell runs, you will see:
* No output other than code progress bar completion

In [ ]:
from streamline.runners.model_runner import ModelExperimentRunner
model_exp = ModelExperimentRunner(
                output_path, experiment_name, algorithms=algorithms, 
                exclude=exclude, class_label=class_label,
                instance_label=instance_label, scoring_metric=primary_metric, 
                metric_direction=metric_direction,
                training_subsample=training_subsample, 
                use_uniform_fi=use_uniform_FI, n_trials=n_trials,
                timeout=timeout, save_plots=False, 
                do_lcs_sweep=do_lcs_sweep, lcs_nu=lcs_nu, lcs_n=lcs_N, 
                lcs_iterations=lcs_iterations,
                lcs_timeout=lcs_timeout, resubmit=False)
model_exp.run(run_parallel=True)

## Phase 6: Statistics Summary and Figure Generation
After cell runs, for each target dataset you will see:
* ROC and PRC plots of CV folds for each algorithm
* An ROC and PRC plot comparing average algorithm performance across CV partitions
* Boxplots for each metric comparing algorithm performance (across CV partitions)
* Top feature importance boxplots for each algorithm (across CV partitions)
* Histogram of feature importance for each algorithm
* Composite feature importance plots

In [ ]:
from streamline.runners.stats_runner import StatsRunner
stats = StatsRunner(output_path, experiment_name, 
                    algorithms=algorithms, exclude=exclude, 
                    class_label=class_label, instance_label=instance_label, 
                    scoring_metric=primary_metric,
                    top_features=top_model_features, sig_cutoff=sig_cutoff, 
                    metric_weight=metric_weight, scale_data=scale_data,
                    plot_roc=plot_ROC, plot_prc=plot_PRC, 
                    plot_fi_box=plot_FI_box, 
                    plot_metric_boxplots=plot_metric_boxplots, 
                    show_plots=True)
stats.run(run_parallel=False)

## Phase 7: Dataset Comparison (Optional: Use only if > 1 dataset was analyzed)
Assuming STREAMLINE was run on more than 1 dataset. After cell runs, for each evaluation metric you will see:
* Boxplots comparing perfomance across analyized target datasets 

In [ ]:
#@title Function to check length for more than one dataset case
def len_datasets(output_path, experiment_name):
    datasets = os.listdir(output_path + '/' + experiment_name)
    remove_list = ['metadata.pickle', 'metadata.csv', 'algInfo.pickle',
                   'jobsCompleted', 'logs', 'jobs', 'DatasetComparisons', 'UsefulNotebooks',
                   experiment_name + '_ML_Pipeline_Report.pdf']
    for text in remove_list:
        if text in datasets:
            datasets.remove(text)
    return len(datasets)

In [ ]:
from streamline.runners.compare_runner import CompareRunner
if len_datasets(output_path, experiment_name) > 1:
    cmp = CompareRunner(output_path, experiment_name, algorithms=algorithms,
                        exclude=exclude, sig_cutoff=sig_cutoff,
                        class_label=class_label, instance_label=instance_label,
                        show_plots=True)
    cmp.run(run_parallel=False)

## Phase 8: PDF Training Report Generator (Optional)
Downloads a PDF report of the analysis

In [ ]:
from streamline.runners.report_runner import ReportRunner
rep = ReportRunner(output_path, experiment_name, 
                   algorithms=algorithms, exclude=exclude)
rep.run(run_parallel=False)

## Phase 9: Apply Models to Replication Data (Optional)

In [ ]:
if applyToReplication:
    from streamline.runners.replicate_runner import ReplicationRunner
    repl = ReplicationRunner(rep_data_path, dataset_for_rep, output_path, 
                             experiment_name, load_algo=True,
                             export_feature_correlations=True, 
                             plot_roc=True, plot_prc=True, plot_metric_boxplots=True)
    repl.run(run_parallel=False)

## Phase 10: PDF Apply Report Generator (Optional)

In [ ]:
if applyToReplication:
    from streamline.runners.report_runner import ReportRunner
    rep = ReportRunner(output_path=output_path, experiment_name=experiment_name,
                       algorithms=algorithms, exclude=exclude, training=False, 
                       rep_data_path=rep_data_path, 
                       dataset_for_rep=dataset_for_rep)
    rep.run(run_parallel=False)

## Phase 11: File Cleanup (Optional)

In [ ]:
from streamline.runners.clean_runner import CleanRunner
clean = CleanRunner(output_path, experiment_name, del_time=del_time, del_old_cv=del_old_cv)
# run_parallel is not used in clean
clean.run()

### Download and Open PDF Summary Report(s)


In [ ]:
from google.colab import files
files.download(output_path + '/' + experiment_name + '/' + experiment_name + '_ML_Pipeline_Report.pdf') 

if applyToReplication:
    dataset_name = dataset_for_rep.split('/')[-1].split('.')[0]
    #from google.colab import files
    pdf_files = []
    for dirpath, dirnames, filenames in os.walk(output_path + '/' + experiment_name 
                                                + '/' + dataset_name + '/applymodel/'):
        for filename in [f for f in filenames if f.endswith(".pdf")]:
            pdf_files.append(os.path.join(dirpath, filename))
    for file_path in pdf_files:
        files.download(file_path) 

# Zip the experiment folder and download.

In [ ]:
experiment_folder = output_path + '/' + experiment_name
!zip -r -q /content/{experiment_name}.zip {experiment_folder}
from google.colab import files
files.download('/content/' + experiment_name + '.zip') 

In [ ]:
#Return notebook to original directory to avoid nested STREAMLINE download bug.
os.chdir(original_wd)